In [1]:
println("Julia version: ", VERSION)

Julia version: 1.7.3


In [2]:
using DataFrames
using Plots
using FreqTables
using TextAnalysis
using Languages

In [3]:
Mystopwords = ["de",  "a",  "o",  "que",  "e",  "do",  "da",  "em",  "um",  "para",  "com",  "não",  "uma",  "os",  "no",  "se",  "na", 
    "por",  "mais",  "as",  "dos",  "como",  "mas",  "ao",  "ele",  "das",  "à",  "seu",  "sua",  "ou",  "quando",  "muito",  "nos",  "já",  
    "eu",  "também",  "só",  "pelo",  "pela",  "até",  "isso",  "ela",  "entre",  "depois",  "sem",  "mesmo",  "aos",  "seus",  "quem", 
    "nas",  "me",  "esse",  "eles",  "você",  "essa",  "num",  "nem",  "suas",  "meu",  "às",  "minha",  "numa",  "pelos",  "elas",  "qual", 
    "nós",  "lhe",  "deles",  "essas",  "esses",  "pelas",  "este",  "dele",  "tu",  "te",  "vocês",  "vos",  "lhes",  "meus",  "minhas", 
    "teu",  "tua",  "teus",  "tuas",  "nosso",  "nossa",  "nossos",  "nossas",  "dela",  "delas",  "esta",  "estes",  "estas",  "aquele", 
    "aquela",  "aqueles",  "aquelas",  "isto",  "aquilo",  "estou",  "está",  "estamos",  "estão",  "estive",  "esteve",  "estivemos", 
    "estiveram",  "estava",  "estávamos",  "estavam",  "estivera",  "estivéramos",  "esteja",  "estejamos",  "estejam",  "estivesse", 
    "estivéssemos",  "estivessem",  "estiver",  "estivermos",  "estiverem",  "hei",  "há",  "havemos",  "hão",  "houve",  "houvemos",  
    "houveram",  "houvera",  "houvéramos",  "haja",  "hajamos",  "hajam",  "houvesse",  "houvéssemos",  "houvessem",  "houver", 
    "houvermos",  "houverem",  "houverei",  "houverá",  "houveremos",  "houverão",  "houveria",  "houveríamos",  "houveriam",  "sou",  
    "somos",  "são",  "era",  "éramos",  "eram",  "fui",  "foi",  "fomos",  "foram",  "fora",  "fôramos",  "seja",  "sejamos",  "sejam", 
    "fosse",  "fôssemos",  "fossem",  "for",  "formos",  "forem",  "serei",  "será",  "seremos",  "serão",  "seria",  "seríamos",  "seriam", 
    "tenho",  "tem",  "temos",  "tém",  "tinha",  "tínhamos",  "tinham",  "tive",  "teve",  "tivemos",  "tiveram",  "tivera",  "tivéramos", 
    "tenha",  "tenhamos",  "tenham",  "tivesse",  "tivéssemos",  "tivessem",  "tiver",  "tivermos",  "tiverem",  "terei",  "terá",  "teremos",
    "terão",  "teria",  "teríamos",  "teriam", "sobre","então","onde","pra","vamos","lá","sim","não","bem","tudo","agora","aqui","é","\\\\",
    "acordar","agora","ainda","aladi","alegrar","além","antar","ante","anthero","antonio","apenas","apesar","apresentação","aquém","araújo","cada","capitar","carioca","carteiro","contra","corpus","corrêa","costa","daquela","demais","diante","edson","entanto","estar","estevar","então","feltrim","final","finar","geral","içar","ie","intuito",
"le","luiz","luzir","mediante","meirelles","mercar","moraes","necessariamente","neto","of","oficiar","oliveira","onde","ora","parir","paulo","pelar","pesar","pilar","pois","primo","quadrar","reinar","res","resinar","reunião","ser","sob","sobre","somente","sr","tal","tais","tanto","thomson","tipo","todo","tony","usecheque","vasconcelos",
"a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z",
"um","dois","três","quatro","cinco","seis","sete","oito","nove","dez","onze","doze","treze","catorze","quinze","dezesseis","dezessete","dezoito","dezenove","vinte",
"aquela","aquelas","aquele","aqueles","àquela","àquelas","daquele","daqueles","daquela","daquelas","naquele","naqueles","naquela","naquelas","neste","nesta","nestes","nestas","nisto","nesse","nessa","nesses","nessas","nisso",
"janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro","mês","meses","ano","anos"]

println("Number of stopwords: ",length(Mystopwords))

Number of stopwords: 381


Create a corpus

In [4]:
listAtas = readdir("../atas");
corpus = []

for i in 1:length(listAtas)
    f = open("../atas/" * listAtas[i])
    txt = readlines(f)
    push!(corpus,txt)
end

println(length(corpus), " atas" )

228 atas


In [5]:
docs = []
for i in 1:length(listAtas)
    fd = FileDocument("../atas/" * listAtas[i]);
    language!(fd, Languages.Portuguese())
    push!(docs,fd)
end
crps = Corpus(docs)
standardize!(crps, StringDocument)
remove_case!(crps)
prepare!(crps, strip_numbers)
prepare!(crps, strip_punctuation)
update_lexicon!(crps)
update_inverse_index!(crps)
crps

A Corpus with 228 documents:
 * 228 StringDocument's
 * 0 FileDocument's
 * 0 TokenDocument's
 * 0 NGramDocument's

Corpus's lexicon contains 11485 tokens
Corpus's index contains 11485 tokens

### char frequency


In [6]:
dict_char = Dict{Char, Int}()
for txt in corpus
    for char in string(txt)
        if haskey(dict_char, char)
         dict_char[char] += 1
      else
         dict_char[char] = 1
      end
    end
end


In [7]:
chars = [];
frequency = [];
for (key, value) in dict_char
    push!(chars,key)
    push!(frequency,value)
end
char_df = DataFrame(Dict("Char"=>chars, "n"=>frequency));
sort!(char_df, [:n, :Char], rev=[true, false])
char_df[1:20,:]

,Char,n
,Any,Any
1,,1206511
2,e,743528
3,a,659885
4,o,614063
5,s,443176
6,r,387184
7,i,371433
8,d,356968
9,n,318473


### word frequency

#### with stop words

In [8]:
dict_words = lexicon(crps);
words = [];
frequency = [];
for (key, value) in dict_words
    push!(words,key)
    push!(frequency,value)
end
words_df = DataFrame(Dict("Words"=>words, "n"=>frequency));
sort!(words_df, [:n, :Words], rev=[true, false])
words_df[1:20,:]

,Words,n
,Any,Any
1,de,93856
2,em,39290
3,a,39225
4,e,32037
5,o,27817
6,do,24770
7,da,21635
8,no,18765
9,para,17303


#### without stop words

In [9]:
crps2 = deepcopy(crps);
prepare!(crps2, strip_stopwords)
prepare!(crps2, strip_numbers)
prepare!(crps2, strip_punctuation)
remove_case!(crps2)
remove_words!(crps2, Mystopwords)
update_lexicon!(crps2)
update_inverse_index!(crps2)
crps2

A Corpus with 228 documents:
 * 228 StringDocument's
 * 0 FileDocument's
 * 0 TokenDocument's
 * 0 NGramDocument's

Corpus's lexicon contains 11018 tokens
Corpus's index contains 11018 tokens

In [10]:
dict_words2 = lexicon(crps2);
words2 = [];
frequency2 = [];
for (key, value) in dict_words2
    push!(words2,key)
    push!(frequency2,value)
end
words_df2 = DataFrame(Dict("Words"=>words2, "n"=>frequency2));
sort!(words_df2, [:n, :Words], rev=[true, false])
words_df2[1:20,:]

,Words,n
,Any,Any
1,preços,7942
2,inflação,7643
3,taxa,4563
4,copom,4001
5,bilhões,3944
6,crescimento,3696
7,produção,3349
8,bens,3326
9,período,3127


### bigram

#### with stop words

In [11]:
dict_bigram = Dict{Any, Any}()
for sent in corpus
    doc = StringDocument(lowercase(string(sent)))
    language!(doc, Languages.Portuguese())
    remove_case!(doc)
    prepare!(doc, strip_numbers)
    prepare!(doc, strip_punctuation)
    dict = ngrams(doc, 2) #this function includes 1-gram and 2-grams
    for k in dict
        if ' ' in k[1] # exclude 1-gram
            if haskey(dict_bigram, k[1])
                dict_bigram[k[1]] += k[2]
            else
                dict_bigram[k[1]] = k[2]
            end
        end
    end
end

In [12]:
bigrams = [];
frequency = [];
for (key, value) in dict_bigram
    push!(bigrams,key)
    push!(frequency,value)
end
bigram_df = DataFrame(Dict("Bigram"=>bigrams, "n"=>frequency));
sort!(bigram_df, [:n, :Bigram], rev=[true, false])
bigram_df[1:20,:]

,Bigram,n
,Any,Any
1,para a,4707
2,e de,4686
3,de em,4524
4,em relação,4106
5,\\ bilhões,3919
6,doze meses,3299
7,de no,2997
8,us \\,2978
9,para o,2776


#### without stop words

In [13]:
dict_bigram = Dict{Any, Any}()
for sent in corpus
    doc = StringDocument(lowercase(string(sent)))
    language!(doc, Languages.Portuguese())
    remove_case!(doc)
    prepare!(doc, strip_numbers)
    prepare!(doc, strip_punctuation)
    prepare!(doc, strip_stopwords)
    remove_words!(doc, Mystopwords)
    dict = ngrams(doc, 2) #this function includes 1-gram and 2-grams
    for k in dict
        if ' ' in k[1] # exclude 1-gram
            if haskey(dict_bigram, k[1])
                dict_bigram[k[1]] += k[2]
            else
                dict_bigram[k[1]] = k[2]
            end
        end
    end
end

In [14]:
bigrams = [];
frequency = [];
for (key, value) in dict_bigram
    push!(bigrams,key)
    push!(frequency,value)
end
bigram_df = DataFrame(Dict("Bigram"=>bigrams, "n"=>frequency));
sort!(bigram_df, [:n, :Bigram], rev=[true, false])
bigram_df[1:20,:]

,Bigram,n
,Any,Any
1,\\ bilhões,3916
2,us \\,2975
3,política monetária,2619
4,banco central,1617
5,bens capital,1228
6,atividade econômica,1089
7,bens consumo,891
8,índice preços,811
9,produção industrial,805


### trigram

#### with stop words

In [15]:
dict_trigram = Dict{Any, Any}()
for sent in corpus
    doc = StringDocument(lowercase(string(sent)))
    language!(doc, Languages.Portuguese())
    remove_case!(doc)
    prepare!(doc, strip_numbers)
    prepare!(doc, strip_punctuation)
    dict = ngrams(doc, 3) #this function includes 1-gram, 2-grams and 3-grams
    for k in dict
        if count( c-> (c == ' ') , k[1]) == 2 # exclude 1-gram and 2-gram
            if haskey(dict_trigram, k[1])
                dict_trigram[k[1]] += k[2]
            else
                dict_trigram[k[1]] = k[2]
            end
        end
    end
end

In [16]:
trigrams = [];
frequency = [];
for (key, value) in dict_trigram
    push!(trigrams,key)
    push!(frequency,value)
end
trigram_df = DataFrame(Dict("Trigram"=>trigrams, "n"=>frequency));
sort!(trigram_df, [:n, :Trigram], rev=[true, false])
trigram_df[1:20,:]

,Trigram,n
,Any,Any
1,em doze meses,2291
2,us \\ bilhões,2002
3,r \\ bilhões,1899
4,de bens de,1647
5,em relação ao,1596
6,em relação a,1328
7,para a inflação,1305
8,bens de capital,1225
9,de acordo com,1225


#### without stop words

In [17]:
dict_trigram = Dict{Any, Any}()
for sent in corpus
    doc = StringDocument(lowercase(string(sent)))
    language!(doc, Languages.Portuguese())
    remove_case!(doc)
    prepare!(doc, strip_numbers)
    prepare!(doc, strip_punctuation)
    prepare!(doc, strip_stopwords)
    remove_words!(doc, Mystopwords)
    dict = ngrams(doc, 3) #this function includes 1-gram, 2-grams and 3-grams
    for k in dict
        if count( c-> (c == ' ') , k[1]) == 2 # exclude 1-gram and 2-gram
            if haskey(dict_trigram, k[1])
                dict_trigram[k[1]] += k[2]
            else
                dict_trigram[k[1]] = k[2]
            end
        end
    end
end

In [18]:
trigrams = [];
frequency = [];
for (key, value) in dict_trigram
    push!(trigrams,key)
    push!(frequency,value)
end
trigram_df = DataFrame(Dict("Trigram"=>trigrams, "n"=>frequency));
sort!(trigram_df, [:n, :Trigram], rev=[true, false])
trigram_df[1:20,:]

,Trigram,n
,Any,Any
1,us \\ bilhões,2001
2,bens consumo duráveis,456
3,us \\ milhões,442
4,produção bens capital,374
5,médias aparadas suavização,330
6,\\ bilhões \\,320
7,bilhões \\ bilhões,293
8,massa salarial real,291
9,dessazonalizados banco central,274
